In [1]:
import h5py
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings

from glob import glob

import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

import rasterio as rio

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap

import plotly.graph_objects as go
# warnings.filterwarnings('ignore')

In [5]:
# !wget -q  https://share.phys.ethz.ch/~pf/albecker/abc/09072022_1154_train.h5
# !wget -q  https://share.phys.ethz.ch/~pf/albecker/abc/09072022_1154_val.h5
# !wget -q https://share.phys.ethz.ch/~pf/albecker/abc/09072022_1154_test.h5

In [2]:
trainset = h5py.File("09072022_1154_train.h5", "r")
validateset = h5py.File("09072022_1154_val.h5", "r")
testset = h5py.File("09072022_1154_test.h5", "r")

In [3]:
trainset.keys()

<KeysViewHDF5 ['agbd', 'cloud', 'images', 'lat', 'lon', 'scl']>

In [4]:
trainset['images'].shape[1:4]

(15, 15, 12)

In [3]:
# train
train_images = np.array(trainset['images'],dtype=np.float64)
train_images = train_images.transpose(0,3,1,2)

train_biomasses = np.array(trainset['agbd'],dtype=np.float64)

# validate
validate_images = np.array(validateset['images'],dtype=np.float64)
validate_images = validate_images.transpose(0,3,1,2)
validate_biomasses = np.array(validateset['agbd'],dtype=np.float64)

# test 
test_images = np.array(testset['images'],dtype=np.float32)
test_images = test_images.transpose(0,3,1,2)
test_biomasses = np.array(testset['agbd'],dtype=np.float32)

In [4]:
print(f"train dataset size {train_images.shape} train lab size {train_biomasses.shape}")
print()
print(f"validate dataset size {validate_images.shape} validate lab size {validate_biomasses.shape}")
print()
print(f"test dataset size {test_images.shape} test lab size {test_biomasses.shape}")

train dataset size (25036, 12, 15, 15) train lab size (25036,)

validate dataset size (5174, 12, 15, 15) validate lab size (5174,)

test dataset size (5190, 12, 15, 15) test lab size (5190,)


In [27]:
import torch
import torch.nn as nn
import torchvision.models as models

# Define the input image size and number of spectral bands
input_size = (12, 15, 15)

# Load a pre-trained ResNet50 model
model = models.resnet50(weights='ResNet50_Weights.DEFAULT')

# Replace the first convolutional layer to accept input with 12 spectral bands
model.conv1 = nn.Conv2d(input_size[0], 64, kernel_size=7, stride=2, padding=0, bias=False)

# Modify the classifier to output the desired number of classes (e.g., 1 for regression)
num_classes = 1
model.fc = nn.Linear(2048, num_classes)

# Define a loss function (e.g., mean squared error)
criterion = nn.MSELoss()

input_ = torch.from_numpy(train_images).float()
output_ = torch.from_numpy(train_biomasses).float().view(-1, 1)

# Pass the input tensor through the model to obtain the predicted output tensor
output_tensor = model(input_)

# Compute the loss between the predicted output tensor and the target tensor
loss = criterion(output_tensor, output_)

# Compute the root mean squared error (RMSE) between the predicted output tensor and the target tensor
rmse = torch.sqrt(loss)

# Print the RMSE value
print('RMSE:', rmse.item())


RMSE: 73.1272964477539


In [28]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim

# Define the input image size and number of spectral bands
input_size = (12, 15, 15)

# Load a pre-trained ResNet50 model
model = models.resnet50(weights='ResNet50_Weights.DEFAULT')

# Replace the first convolutional layer to accept input with 12 spectral bands
model.conv1 = nn.Conv2d(input_size[0], 64, kernel_size=7, stride=2, padding=3, bias=False)

# Modify the classifier to output the desired number of classes (e.g., 1 for regression)
num_classes = 1
model.fc = nn.Linear(2048, num_classes)

# Define a loss function (e.g., mean squared error)
criterion = nn.MSELoss()

# Define an optimizer (e.g., Adam) and a learning rate
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convert the training data to tensors
input_ = torch.from_numpy(train_images).float()
output_ = torch.from_numpy(train_biomasses).float().view(-1, 1)

# Define the number of epochs
num_epochs = 10

# Train the model
for epoch in range(num_epochs):
    # Set the model to training mode
    model.train()

    # Zero the gradients
    optimizer.zero_grad()

    # Pass the input tensor through the model to obtain the predicted output tensor
    output_tensor = model(input_)

    # Compute the loss between the predicted output tensor and the target tensor
    loss = criterion(output_tensor, output_)

    # Backpropagate the gradients
    loss.backward()

    # Update the parameters
    optimizer.step()

    # Compute the root mean squared error (RMSE) between the predicted output tensor and the target tensor
    rmse = torch.sqrt(loss)

    # Print the RMSE value for each epoch
    print('Epoch [{}/{}], RMSE: {:.4f}'.format(epoch+1, num_epochs, rmse.item()))


Epoch [1/10], RMSE: 73.1131
Epoch [2/10], RMSE: 72.9937
Epoch [3/10], RMSE: 72.2937
Epoch [4/10], RMSE: 71.2826
Epoch [5/10], RMSE: 69.7239
Epoch [6/10], RMSE: 68.2044
Epoch [7/10], RMSE: 67.1426
Epoch [8/10], RMSE: 66.2390
Epoch [9/10], RMSE: 64.4003
Epoch [10/10], RMSE: 63.1294


In [24]:
import torch
import torch.nn as nn
import torchvision.models as models
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

# Define the input size and image dimensions
image_size = (12,15, 15)

# Define the ResNet50 model and modify its input layer to accept 12 spectral bands
resnet = models.resnet50(weights='ResNet50_Weights.DEFAULT')
resnet.conv1 = nn.Conv2d(image_size[0], 64, kernel_size=7, stride=2, padding=0, bias=False)

# Freeze all layers except for the last one
for param in resnet.parameters():
    param.requires_grad = False
resnet.fc.requires_grad = True

# Define the MLPRegressor with one hidden layer of 100 units
mlp = MLPRegressor(hidden_layer_sizes=(500,), max_iter=1000)

# Use the ResNet50 to extract features from the input images
def get_features(input_tensor):
    resnet.eval()
    with torch.no_grad():
        features = resnet(input_tensor)
    return features

# Extract features from the training input images
train_features = get_features(torch.from_numpy(train_images).float())

# Train the MLPRegressor using RMSE loss
mlp.fit(train_features.numpy(), train_biomasses)

# Extract features from the test input images and use the MLPRegressor to predict the output
test_features = get_features(torch.from_numpy(validate_images).float())
y_pred = mlp.predict(test_features.numpy())

# Compute the RMSE loss between the predicted output and the target output
rmse = np.sqrt(mean_squared_error(validate_biomasses, y_pred))

# Print the RMSE loss value
print('RMSE:', rmse)


RMSE: 65.59846255300816


In [ ]:
# s2_images_h5 = h5py.File("images_test.h5", "r")
# #prepare test set sentinel 2 images 
# s2_images = np.array(s2_images_h5["images"])
# s2_images = s2_images.transpose(0,3,1,2)

## predict on giz test data
# pred_giz = pipe.predict(s2_images)
# ID_S2_pair = pd.read_csv('/content/UniqueID-SentinelPair.csv')

# preds = pd.DataFrame({'Target':pred_giz}).rename_axis('S2_idx').reset_index()
# preds = ID_S2_pair.merge(preds, on='S2_idx').drop(columns=['S2_idx'])
# preds
## preds.to_csv('GIZ_Biomass_predictions.csv', index=False)